In [1]:
from pymor.core.defaults import set_defaults, print_defaults
set_defaults({'pymor.discretizers.builtin.gui.jupyter.get_visualizer.backend': 'pyvista'})

In [2]:
"""Simplified version of the thermalblock demo.

Usage:
  thermalblock_simple.py MODEL ALG SNAPSHOTS RBSIZE TEST

Arguments:
"""

from typer import Argument, run

from pymor.basic import *
from pymor.tools.typer import Choices


# parameters for high-dimensional models
XBLOCKS = 2             # pyMOR/FEniCS
YBLOCKS = 2             # pyMOR/FEniCS
GRID_INTERVALS = 100    # pyMOR/FEniCS
FENICS_ORDER = 2
NGS_ORDER = 2
TEXT = 'pyMOR'


####################################################################################################
# Main script                                                                                      #
####################################################################################################

def main(
    model: Choices('pymor fenics ngsolve pymor_text') = Argument(..., help='High-dimensional model.'),
):
    # discretize
    ############
    if model == 'pymor':
        return discretize_pymor()
    elif model == 'fenics':
        return discretize_fenics()
    elif model == 'ngsolve':
        return discretize_ngsolve()
    elif model == 'pymor_text':
        return discretize_pymor_text()
    else:
        raise NotImplementedError


####################################################################################################
# High-dimensional models                                                                          #
####################################################################################################


def discretize_pymor():

    # setup analytical problem
    problem = thermal_block_problem(num_blocks=(XBLOCKS, YBLOCKS))

    # discretize using continuous finite elements
    fom, _ = discretize_stationary_cg(problem, diameter=1. / GRID_INTERVALS)

    return fom, problem.parameter_space


def discretize_fenics():
    from pymor.tools import mpi

    if mpi.parallel:
        from pymor.models.mpi import mpi_wrap_model
        fom = mpi_wrap_model(_discretize_fenics, use_with=True, pickle_local_spaces=False)
    else:
        fom = _discretize_fenics()
    return fom, fom.parameters.space((0.1, 1))


def _discretize_fenics():

    # assemble system matrices - FEniCS code
    ########################################

    import dolfin as df

    mesh = df.UnitSquareMesh(GRID_INTERVALS, GRID_INTERVALS, 'crossed')
    V = df.FunctionSpace(mesh, 'Lagrange', FENICS_ORDER)
    u = df.TrialFunction(V)
    v = df.TestFunction(V)

    diffusion = df.Expression('(lower0 <= x[0]) * (open0 ? (x[0] < upper0) : (x[0] <= upper0)) *'
                              '(lower1 <= x[1]) * (open1 ? (x[1] < upper1) : (x[1] <= upper1))',
                              lower0=0., upper0=0., open0=0,
                              lower1=0., upper1=0., open1=0,
                              element=df.FunctionSpace(mesh, 'DG', 0).ufl_element())

    def assemble_matrix(x, y, nx, ny):
        diffusion.user_parameters['lower0'] = x/nx
        diffusion.user_parameters['lower1'] = y/ny
        diffusion.user_parameters['upper0'] = (x + 1)/nx
        diffusion.user_parameters['upper1'] = (y + 1)/ny
        diffusion.user_parameters['open0'] = (x + 1 == nx)
        diffusion.user_parameters['open1'] = (y + 1 == ny)
        return df.assemble(df.inner(diffusion * df.nabla_grad(u), df.nabla_grad(v)) * df.dx)

    mats = [assemble_matrix(x, y, XBLOCKS, YBLOCKS)
            for x in range(XBLOCKS) for y in range(YBLOCKS)]
    mat0 = mats[0].copy()
    mat0.zero()
    h1_mat = df.assemble(df.inner(df.nabla_grad(u), df.nabla_grad(v)) * df.dx)

    f = df.Constant(1.) * v * df.dx
    F = df.assemble(f)

    bc = df.DirichletBC(V, 0., df.DomainBoundary())
    for m in mats:
        bc.zero(m)
    bc.apply(mat0)
    bc.apply(h1_mat)
    bc.apply(F)

    # wrap everything as a pyMOR model
    ##################################

    # FEniCS wrappers
    from pymor.bindings.fenics import FenicsVectorSpace, FenicsMatrixOperator, FenicsVisualizer

    # define parameter functionals (same as in pymor.analyticalproblems.thermalblock)
    parameter_functionals = [ProjectionParameterFunctional('diffusion',
                                                           size=YBLOCKS*XBLOCKS,
                                                           index=YBLOCKS - y - 1 + x*YBLOCKS)
                             for x in range(XBLOCKS) for y in range(YBLOCKS)]

    # wrap operators
    ops = [FenicsMatrixOperator(mat0, V, V)] + [FenicsMatrixOperator(m, V, V) for m in mats]
    op = LincombOperator(ops, [1.] + parameter_functionals)
    rhs = VectorOperator(FenicsVectorSpace(V).make_array([F]))
    h1_product = FenicsMatrixOperator(h1_mat, V, V, name='h1_0_semi')

    # build model
    visualizer = FenicsVisualizer(FenicsVectorSpace(V))
    fom = StationaryModel(op, rhs, products={'h1_0_semi': h1_product},
                          visualizer=visualizer)

    return fom


def discretize_ngsolve():
    from ngsolve import (ngsglobals, Mesh, H1, CoefficientFunction, LinearForm, SymbolicLFI,
                         BilinearForm, SymbolicBFI, grad, TaskManager)
    from netgen.csg import CSGeometry, OrthoBrick, Pnt
    import numpy as np

    ngsglobals.msg_level = 1

    geo = CSGeometry()
    obox = OrthoBrick(Pnt(-1, -1, -1), Pnt(1, 1, 1)).bc("outer")

    b = []
    b.append(OrthoBrick(Pnt(-1, -1, -1), Pnt(0.0, 0.0, 0.0)).mat("mat1").bc("inner"))
    b.append(OrthoBrick(Pnt(-1,  0, -1), Pnt(0.0, 1.0, 0.0)).mat("mat2").bc("inner"))
    b.append(OrthoBrick(Pnt(0,  -1, -1), Pnt(1.0, 0.0, 0.0)).mat("mat3").bc("inner"))
    b.append(OrthoBrick(Pnt(0,   0, -1), Pnt(1.0, 1.0, 0.0)).mat("mat4").bc("inner"))
    b.append(OrthoBrick(Pnt(-1, -1,  0), Pnt(0.0, 0.0, 1.0)).mat("mat5").bc("inner"))
    b.append(OrthoBrick(Pnt(-1,  0,  0), Pnt(0.0, 1.0, 1.0)).mat("mat6").bc("inner"))
    b.append(OrthoBrick(Pnt(0,  -1,  0), Pnt(1.0, 0.0, 1.0)).mat("mat7").bc("inner"))
    b.append(OrthoBrick(Pnt(0,   0,  0), Pnt(1.0, 1.0, 1.0)).mat("mat8").bc("inner"))
    box = (obox - b[0] - b[1] - b[2] - b[3] - b[4] - b[5] - b[6] - b[7])

    geo.Add(box)
    for bi in b:
        geo.Add(bi)
    # domain 0 is empty!

    mesh = Mesh(geo.GenerateMesh(maxh=0.3))

    # H1-conforming finite element space
    V = H1(mesh, order=NGS_ORDER, dirichlet="outer")
    v = V.TestFunction()
    u = V.TrialFunction()

    # Coeff as array: variable coefficient function (one CoefFct. per domain):
    sourcefct = CoefficientFunction([1 for i in range(9)])

    with TaskManager():
        # the right hand side
        f = LinearForm(V)
        f += SymbolicLFI(sourcefct * v)
        f.Assemble()

        # the bilinear-form
        mats = []
        coeffs = [[0, 1, 0, 0, 0, 0, 0, 0, 1],
                  [0, 0, 1, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 1, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 1, 1, 0, 0, 0]]
        for c in coeffs:
            diffusion = CoefficientFunction(c)
            a = BilinearForm(V, symmetric=False)
            a += SymbolicBFI(diffusion * grad(u) * grad(v), definedon=(np.where(np.array(c) == 1)[0] + 1).tolist())
            a.Assemble()
            mats.append(a.mat)

    from pymor.bindings.ngsolve import NGSolveVectorSpace, NGSolveMatrixOperator, NGSolveVisualizer

    space = NGSolveVectorSpace(V)
    op = LincombOperator([NGSolveMatrixOperator(m, space, space) for m in mats],
                         [ProjectionParameterFunctional('diffusion', len(coeffs), i) for i in range(len(coeffs))])

    h1_0_op = op.assemble(Mu(diffusion=[1] * len(coeffs))).with_(name='h1_0_semi')

    F = space.zeros()
    F._list[0].real_part.impl.vec.data = f.vec
    F = VectorOperator(F)

    fom = StationaryModel(op, F, visualizer=NGSolveVisualizer(mesh, V),
                          products={'h1_0_semi': h1_0_op})

    return fom, fom.parameters.space((0.1, 1))


def discretize_pymor_text():

    # setup analytical problem
    problem = text_problem(TEXT)

    # discretize using continuous finite elements
    fom, _ = discretize_stationary_cg(problem, diameter=1.)

    return fom, problem.parameter_space



In [3]:
pymor_model, _ = main('pymor')
U = pymor_model.solve([1, 1, 1, 1])
pymor_model.visualize(U, title='pymor')

Accordion(children=(HTML(value='', layout=Layout(height='16em', overflow_y='auto', width='100%')),), selected_…

AppLayout(children=(VBox(children=(HTML(value='<h3>Data</h3>'), Dropdown(description='Colormap:', options={'Br…

In [5]:
ngsolve_model, _ = main('ngsolve')
U = ngsolve_model.solve([1, 1, 1, 1])
ngsolve_model.visualize(U, title='ngsolve', filename='/pymor/ngsolve.vtk')
ngsolve_model.visualize(U, title='ngsolve')

importing NGSolve-6.2.2101
 Start Findpoints
 Find edges
 Start Findpoints
 Find edges
 Start Findpoints
 Find edges
 Surface 1 / 36
 Surface 2 / 36
 Surface 3 / 36
 Surface 4 / 36
 Surface 5 / 36
 Surface 6 / 36
 Surface 7 / 36
 Surface 8 / 36
 Surface 9 / 36
 Surface 10 / 36
 Surface 11 / 36
 Surface 12 / 36
 Surface 13 / 36
 Surface 14 / 36
 Surface 15 / 36
 Surface 16 / 36
 Surface 17 / 36
 Surface 18 / 36
 Surface 19 / 36
 Surface 20 / 36
 Surface 21 / 36
 Surface 22 / 36
 Surface 23 / 36
 Surface 24 / 36
 Surface 25 / 36
 Surface 26 / 36
 Surface 27 / 36
 Surface 28 / 36
 Surface 29 / 36
 Surface 30 / 36
 Surface 31 / 36
 Surface 32 / 36
 Surface 33 / 36
 Surface 34 / 36
 Surface 35 / 36
 Surface 36 / 36
 Meshing subdomain 1 of 9
 Meshing subdomain 2 of 9
 Delaunay meshing
 start tetmeshing
 Success !
 363 points, 168 elements
 Meshing subdomain 3 of 9
 Delaunay meshing
 start tetmeshing
 Success !
 365 points, 336 elements
 Meshing subdomain 4 of 9
 Delaunay meshing
 start tetme

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )
used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )
used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )
used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


Accordion(children=(HTML(value='', layout=Layout(height='16em', overflow_y='auto', width='100%')),), selected_…

AppLayout(children=(VBox(children=(HTML(value='<h3>VectorArray0</h3>'), Dropdown(description='Colormap:', opti…

In [4]:
fenics_model, _ = main('fenics')
U = fenics_model.solve([1, 1, 1, 1])
fenics_model.visualize(U, title='fenics', filename='/pymor/fenics.pvd')
# currently fails due to mismatched hdf5 library versions between system and h5py wheel
fenics_model.visualize(U, title='fenics')

Accordion(children=(HTML(value='', layout=Layout(height='16em', overflow_y='auto', width='100%')),), selected_…

/home/pymor/.local/lib/python3.9/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.10.4 when it was built against 1.12.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


ValueError: High bound is not valid (high bound is not valid)